# **Install and Import Libraries**

> ##### **Add the OpenAI API key in config/secrets.env file as follows:**

> ###### **OPENAI_API_KEY = "<api_key>"**


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from dotenv import load_dotenv
import os
import json
from neo4j import GraphDatabase

# load config
load_dotenv("../config/config.env")

# load secrets
load_dotenv("../config/secrets.env")

from data_pipeline import *

# **1. Scrape Website**

In [7]:
df = scrape_website()

Scraping in progress...
 ├─ Depth: 1. Found 200 items. Heading: Sammanträden
	 ├─ Depth: 2. Found 19 items. Heading: Mötets uppgifter: Stadsstyrelsen 17/2023, Protokol...
		 ├─ Depth: 3. Found 9 items. Heading: Ärende:Utlåtande om utkast till investeringsprogra...
		 ├─ Depth: 3. Found 4 items. Heading: Ärende:Utlåtande över ansökan om undantag; 893-419...
		 ├─ Depth: 3. Found 5 items. Heading: Ärende:Utlåtande över ansökan om undantag; 893-404...
		 ├─ Depth: 3. Found 2 items. Heading: Ärende:Finansiering av allmänna vägars ytbeläggnin...
		 ├─ Depth: 3. Found 2 items. Heading: Ärende:Rapport om byggnadsbeståndets värden och ny...
		 ├─ Depth: 3. Found 2 items. Heading: Ärende:Beviljande av bidrag för allmännyttiga samf...
		 ├─ Depth: 3. Found 2 items. Heading: Ärende:Beviljande av bidrag för allmännyttiga samf...
		 ├─ Depth: 3. Found 2 items. Heading: Ärende:Rusmedelsförebyggande plan
	 ├─ Depth: 2. Found 6 items. Heading: Mötets uppgifter: Personalsektionen 6/2023, Protok...
	 ├─

In [10]:
df

,doc_link,title,section,meeting_date,meeting_time,meeting_reference,body,parent_link,start_time
0,https://kungorelse.nykarleby.fi:8443/ktwebbin/...,Sammanträdesuppgifter och deltagande,§ 0,18.12.2023,,17/2023,Stadsstyrelsen,,16:46
1,https://kungorelse.nykarleby.fi:8443/ktwebbin/...,Sammanträdets konstituerande,§ 278,18.12.2023,,17/2023,Stadsstyrelsen,,16:46
2,https://kungorelse.nykarleby.fi:8443/ktwebbin/...,Anmälningsärenden,§ 279,18.12.2023,,17/2023,Stadsstyrelsen,,16:46
3,https://kungorelse.nykarleby.fi:8443/ktwebbin/...,Utlåtande om utkast till investeringsprogramme...,§ 280,18.12.2023,,17/2023,Stadsstyrelsen,,16:46
4,https://kungorelse.nykarleby.fi:8443/ktwebbin/...,"Begäran om utlåtande, Trafikledsverket",,18.12.2023,,17/2023,Stadsstyrelsen,https://kungorelse.nykarleby.fi:8443/ktwebbin/...,16:46
...,...,...,...,...,...,...,...,...,...
4657,https://kungorelse.nykarleby.fi:8443/ktwebbin/...,Sammanträdets laglighet och beslutförhet,§ 1,17.1.2022,,1/2022,Ungdomsfullmäktige,,15:45
4658,https://kungorelse.nykarleby.fi:8443/ktwebbin/...,Val av protokolljusterare,§ 2,17.1.2022,,1/2022,Ungdomsfullmäktige,,15:45
4659,https://kungorelse.nykarleby.fi:8443/ktwebbin/...,Godkännande av föredragningslistan,§ 3,17.1.2022,,1/2022,Ungdomsfullmäktige,,15:45
4660,https://kungorelse.nykarleby.fi:8443/ktwebbin/...,"Sommarbete 2022, Hur gör vi med sommarjobbs-ji...",§ 4,17.1.2022,,1/2022,Ungdomsfullmäktige,,15:45


# **2. Download all PDFs from links**

In [17]:
df = download_documents()

c:\users\ashdah\desktop\democracy-chatbot\src\data_pipeline\document_downloader.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df = download_pdfs(df, PROTOCOLS_PDF_PATH)


In [ ]:
df

# **3. Extract HTML from PDFs**

In [ ]:
convert_pdf_to_html()

# **4. Extract Meeting Metadata from PDF with LLM**

In [ ]:
df = extract_meeting_metadata()

In [ ]:
df

# **5. Extract Proposals and Decisions**

## still not working as expected

In [ ]:
df = extract_meeting_agenda(10)

# **6. Export JSON**

In [ ]:
def construct_individual_json(df):
    """
    Constructs the individual json files for each meeting item and meeting metadata
    """
    for index, row in df.iterrows():
        # create the path to the save the json files
        save_path = os.path.join(os.getenv("EXTRACTED_JSON_PATH"), row['verksamhetsorgan'], row['meeting_date'], row['doc_name'].split(".")[0])

        if row['meeting_end_time'] != "":

            # ensure that the path exists
            os.makedirs(os.path.dirname(save_path), exist_ok=True)

            # path to save the metadata json file
            metadata_save_path = os.path.join(os.path.dirname(save_path), "llm_meeting_metadata.json")

            json_data = f'''{{
                    "meeting_date": "{row['meeting_date']}",
                    "start_time": "{row['start_time']}",
                    "meeting_reference": "{row['meeting_reference']}",
                    "end_time": "{row['end_time']}",
                    "meeting_location": "{row['meeting_location']}",
                    "participants": {row['participants']},
                    "substitutes": {row['substitutes']},
                    "additional_attendees": {row['additional_attendees']},
                    "signed_by": {row['signed_by']},
                    "adjusted_by": {row['adjusted_by']},
                    "adjustment_date": "{row['adjustment_date']}",
                    "meeting_items": [] }}''' # meeting items is added when constructing the aggregate JSON file

            json_data = json.dumps(json.loads(json_data), indent=4, ensure_ascii=False)
            # save the metadata json file
            with open(metadata_save_path, "w") as f:
                f.write(json_data)
                
        elif row['agenda_metadata'] != "":
            # ensure that the path exists
            os.makedirs(save_path, exist_ok=True)

            # path to save the meeting item json file
            item_save_path = os.path.join(save_path, "llm_meeting_item.json")

            item = json.loads(row['agenda_metadata'])
            item['title'] = row['title']
            item['section'] = row['section']

            # get all the atachments of the row based on parent link
            attachments = df[df['parent_link'] == row['doc_link']]

            # add the attachments to the item
            item['attachments'] = []
            for index, attachment in attachments.iterrows():
                item['attachments'].append({
                    "title": attachment['title'],
                    "link": attachment['doc_link']
                })
            # save the meeting item json file
            with open(item_save_path, "w") as f:
                f.write(json.dumps(item, indent=4, ensure_ascii=False))

In [4]:
df = pd.read_csv("../data/metadata.csv", index_col=0)
df.fillna("", inplace=True)

In [ ]:
construct_individual_json(df)

In [ ]:
def construct_aggregate_json(extracted_json_path, construct_from = "llm"): # construct_from = "llm" or "manual"
    """
    Constructs a single JSON out of all the meeting metadata and items
    """
    aggregate_json = {}
    aggregate_json['verksamhetsorgan'] = []
    if not os.path.exists(extracted_json_path):
        print("Extracted JSON path does not exist")
        return
    for organ in os.scandir(extracted_json_path):
        if not organ.is_dir():
            continue
        aggregate_meeting = []
        for meeting in os.scandir(organ.path):
            metadata_path = os.path.join(meeting.path, f"{construct_from}_meeting_metadata.json")
            if os.path.exists(metadata_path):
                with open(metadata_path, "r") as f:
                    metadata = json.load(f)
                for item in os.scandir(meeting.path):
                    if item.is_dir():
                        item_path = os.path.join(item.path, f"{construct_from}_meeting_item.json")
                        if os.path.exists(item_path):
                            with open(item_path, "r") as f:
                                item = json.load(f)
                            metadata['meetingItems'].append(item)
                aggregate_meeting.append(metadata)
        aggregate_json['verksamhetsorgan'].append({
            "name": organ.name,
            "meetings": aggregate_meeting
        })

    with open(os.path.join(extracted_json_path, f"{construct_from}_aggregate_data.json"), "w") as f:
        f.write(json.dumps(aggregate_json, indent=4, ensure_ascii=False))

In [ ]:
construct_aggregate_json(os.getenv("EXTRACTED_JSON_PATH"), construct_from = "llm") # construct_from = "llm" or "manual"

In [ ]:

# sanity check for number of folders created. should correspond to number of meetings with metadata extracted with llm
import glob
len(glob.glob(os.getenv("EXTRACTED_JSON_PATH") + "/*/*/*"))

# **7. Create a Knowledge Graph from JSON**

In [ ]:
# setup cohere api for embedding
import cohere
from tqdm import tqdm

In [ ]:
def generate_embeddings(texts):
    """
    Generates embeddings for the input texts
    """
    co = cohere.Client(os.getenv("COHERE_API_KEY"))
    response = co.embed(texts=texts, model='embed-multilingual-v3.0', input_type="search_document")  
    return response.embeddings


In [ ]:
# Function to execute Cypher queries
def execute_cypher_queries(driver, data):

    with driver.session() as session:
        organ_embeddings = generate_embeddings([organ.get("name", "") for organ in data.get("verksamhetsorgan", [])])
        for i, organ in enumerate(tqdm(data.get("verksamhetsorgan", []), position=0, desc="Creating organs")):
            # Merge Organ
            organ_name = organ.get("name", "")
            session.run("""
                MERGE (o:Organ {name: $organ_name})
                SET o.name_embedding = $name_embedding
                """, 
                organ_name=organ_name,
                name_embedding=organ_embeddings[i])

            # Process meetings
            meeting_embeddings = generate_embeddings([meeting.get("meetingPlace", "") for meeting in organ.get("meetings", [])])
            for j, meeting in enumerate(tqdm(organ.get("meetings", []), position=1, leave=False, desc="Creating meetings")):
                # Merge Meeting
                meeting_place = meeting.get("meetingPlace", "")
                meeting_id = session.run("""
                    MERGE (m:Meeting {
                    meeting_date: $meeting_date,
                    start_time: $start_time,
                    meeting_reference: $meeting_reference,
                    end_time: $end_time,
                    meeting_place: $meeting_place
                    })
                    WITH m
                    MATCH (o:Organ {name: $organ_name})
                    MERGE (o)-[:HOSTED]->(m)
                    SET m.meeting_place_embedding = $meeting_place_embedding
                    RETURN id(m)
                    """, 
                    meeting_date=meeting.get("meetingDate", ""),
                    start_time=meeting.get("startTime", ""),
                    meeting_reference=meeting.get("meetingReference", ""),
                    end_time=meeting.get("endTime", ""),
                    meeting_place=meeting_place,
                    organ_name=organ_name,
                    meeting_place_embedding=meeting_embeddings[j]
                    ).single()[0]

                # Process Members
                for person in meeting.get("members", []):
                    session.run("""
                        MERGE (p:Person {name: $name})
                        WITH p
                        MATCH (m:Meeting) WHERE id(m) = $meeting_id
                        MERGE (p)-[:ATTENDED {
                        role: $role, 
                        attendance: coalesce($attendance, '')
                        }]->(m)
                        """, 
                        name=person.get("name", ""),
                        role=person.get("role", ""),
                        attendance=person.get("attendance", ""),
                        meeting_id=meeting_id)

                # Process Substitutes
                for substitute in meeting.get("substitutes", []):
                    session.run("""
                        // Create or find the substitute node and connect to the meeting
                        MERGE (s:Person {name: $name})
                        WITH s
                        MATCH (m:Meeting) WHERE id(m) = $meeting_id
                        MERGE (s)-[:SUBSTITUTE_ATTENDEE]->(m)
                        WITH s
                        // Only proceed if substitutedFor is not an empty string
                        WHERE $substituted_for <> ''
                        // Create or find the substituted person node and create a relationship
                        MERGE (substituted:Person {name: $substituted_for})
                        MERGE (s)-[:SUBSTITUTED_FOR]->(substituted)
                    """, name=substitute.get("name", ""),
                        substituted_for=substitute.get("substitutedFor", ""),
                        meeting_id=meeting_id)

                # Process Additional Attendees
                for attendee in meeting.get("additionalAttendees", []):
                    session.run("""
                        MERGE (a:Person {name: $name})
                        WITH a
                        MATCH (m:Meeting) WHERE id(m) = $meeting_id
                        MERGE (a)-[:ADDITIONAL_ATTENDEE {
                            role: coalesce($role, '')
                        }]->(m)
                    """, 
                        name=attendee.get("name", ""),
                        role=attendee.get("role", ""),
                        meeting_id=meeting_id)

                # Process Meeting Items
                for item in meeting.get("meetingItems", []):
                    item_embeddings = generate_embeddings([
                        item.get("rubrik", ""),
                        item.get("protocolContext", ""),
                        item.get("beredareDescription", ""),
                        item.get("förslagDescription", ""),
                        item.get("beslut", "")
                    ])
                    item_id = session.run("""
                        MERGE (i:MeetingItem {
                            rubrik: coalesce($rubrik, ''),
                            section: coalesce($section, ''),
                            protocol_context: coalesce($protocol_context, ''),
                            beslut: coalesce($beslut, '')
                        })
                        WITH i
                        MATCH (m:Meeting) WHERE id(m) = $meetingId
                        MERGE (m)-[:HAS_ITEM]->(i)
                        SET i.rubrik_embedding = $rubrik_embedding,
                            i.protocol_context_embedding = $protocol_context_embedding,
                            i.beredare_description_embedding = $beredare_description_embedding,
                            i.förslag_description_embedding = $förslag_description_embedding,
                            i.beslut_embedding = $beslut_embedding
                        RETURN id(i)
                        """, 
                        rubrik=item.get("rubrik", ""),
                        section=item.get("section", ""),
                        meetingId=meeting_id,
                        protocol_context=item.get("protocolContext", ""),
                        beredare_description=item.get("beredareDescription", ""),
                        förslag_description = item.get("förslagDescription", ""),
                        beslut=item.get("beslut", ""),
                        rubrik_embedding=item_embeddings[0],
                        protocol_context_embedding=item_embeddings[1],
                        beredare_description_embedding=item_embeddings[2],
                        förslag_description_embedding=item_embeddings[3],
                        beslut_embedding=item_embeddings[4]
                        ).single()[0]

                    # # Link Meeting to Meeting Item
                    # session.run("""
                    #     MATCH (m:Meeting) WHERE id(m) = $meetingId
                    #     MATCH (i:MeetingItem) WHERE id(i) = $itemId
                    #     MERGE (m)-[:HAS_ITEM]->(i)
                    # """, meetingId=meeting_id, itemId=item_id)

                    # Process Preparers and Proposers similarly inside Meeting Items
                    for preparer in item.get("preparers", []):
                        session.run("""
                            MERGE (p:Person {name: coalesce($name, '')})
                            WITH p
                            MATCH (i:MeetingItem) WHERE id(i) = $item_id
                            MERGE (p)-[:PREPARED]->(i)
                            """, 
                            name=preparer.get("name", ""),
                            item_id=item_id)
                        
                    for proposer in item.get("proposers", []):
                        session.run("""
                            MERGE (p:Person {name: coalesce($name, '')})
                            WITH p
                            MATCH (i:MeetingItem) WHERE id(i) = $item_id
                            MERGE (p)-[:PROPOSED]->(i)
                            """, 
                            name=proposer.get("name", ""),
                            item_id=item_id)
                        
                    # Process Signatories
                    for signatory in item.get("signatories", []):
                        session.run("""
                            MERGE (s:Person {name: coalesce($name, '')})
                            WITH s
                            MATCH (i:MeetingItem) WHERE id(i) = $item_id
                            MERGE (s)-[:SIGNED]->(i)
                            """, 
                            name=signatory.get("name", ""),
                            item_id=item_id)
                        
                    # Process Adjusters
                    for adjuster in item.get("adjusters", []):
                        session.run("""
                            MERGE (a:Person {name: coalesce($name, '')})
                            WITH a
                            MATCH (i:MeetingItem) WHERE id(i) = $item_id
                            MERGE (a)-[:ADJUSTED]->(i)
                            """, 
                            name=adjuster.get("name", ""),
                            item_id=item_id)
                        
                    # Process Meeting Item Attachments
                    attachment_embeddings = generate_embeddings([attachment.get("rubrik", "") for attachment in item.get("attachments", [])])
                    for k, attachment in enumerate(item.get("attachments", [])):
                        session.run("""
                            MERGE (a:Attachment {link: coalesce($link, ''), rubrik: coalesce($rubrik, '')})
                            WITH a
                            MATCH (i:MeetingItem) WHERE id(i) = $item_id
                            MERGE (i)-[:HAS_ATTACHMENT]->(a)
                            SET a.rubrik_embedding = $rubrik_embedding
                            """, 
                            link=attachment.get("link", ""),
                            rubrik=attachment.get("rubrik", ""),
                            rubrik_embedding=attachment_embeddings[k],
                            item_id=item_id)

def create_embeddings_index(driver):
    with driver.session() as session:
            # define options for cypher query
            options = """OPTIONS {indexConfig: {
                    `vector.dimensions`: 1024,
                    `vector.similarity_function`: 'cosine'}}"""

            # create vector index for each embedding
            session.run(f"""
                CREATE VECTOR INDEX `organ-name-embedding` IF NOT EXISTS
                FOR (n:Organ) ON (n.name_embedding)
                {options}
            """)
            session.run(f"""
                CREATE VECTOR INDEX `meeting-place-embedding` IF NOT EXISTS
                FOR (n:Meeting) ON (n.meeting_place_embedding)
                {options}
            """)

            item_properties = ["rubrik_embedding", "protocol_context_embedding", "beredare_description_embedding", "förslag_description_embedding", "beslut_embedding"]
            for property in item_properties:
                session.run(f"""
                    CREATE VECTOR INDEX `item_{property}` IF NOT EXISTS
                    FOR (n:MeetingItem) ON (n.{property})
                    {options}
                """)
                
            session.run(f"""
                CREATE VECTOR INDEX `attachment-rubrik-embedding` IF NOT EXISTS
                FOR (n:Attachment) ON (n.rubrik_embedding)
                {options}
            """)

def get_index_info(driver):
    with driver.session() as session:
        res = session.run("""
                        SHOW VECTOR INDEXES YIELD name, labelsOrTypes, properties
                            """)
        return res.to_df().to_markdown()


def create_knowledge_graph(constuct_from = "llm"): # construct_from = "llm" or "manual"
    # Load JSON data
    aggregate_json_path = os.path.join(os.getenv("EXTRACTED_JSON_PATH"), f"{constuct_from}_aggregate_data.json")

    with open(aggregate_json_path, "r") as f:
        data = json.load(f)

    # Neo4j connection details
    uri = os.getenv("NEO4J_URI")
    username = os.getenv("NEO4J_USERNAME")
    password = os.getenv("NEO4J_PASSWORD")

    # Connect to Neo4j
    driver = GraphDatabase.driver(uri, auth=(username, password))

    # Execute Cypher queries
    execute_cypher_queries(driver, data)

    # Create embeddings index
    create_embeddings_index(driver)

In [ ]:
create_knowledge_graph(constuct_from = "llm") # construct_from = "llm" or "manual"

## **8. Test Knowledge Graph Query with LLM**

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain_core.prompts.prompt import PromptTemplate
import os

In [ ]:
graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"], username=os.environ["NEO4J_USERNAME"], password=os.environ["NEO4J_PASSWORD"]
)

In [ ]:
graph.schema

In [ ]:
# Neo4j connection details
url = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

# Connect to Neo4j
driver = GraphDatabase.driver(url, auth=(username, password))

In [ ]:
index_info = get_index_info(driver)
CYPHER_GENERATION_TEMPLATE = f"""Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{{schema}}
-----
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
-----
For matching properties with that can use semantic similarity, use the following cypher syntax to retrieve nodes from a vector index:

CALL db.index.vector.queryNodes(indexName :: STRING, numberOfNearestNeighbours :: INTEGER, query :: STRING) :: (node :: NODE, score :: FLOAT)

Query is the search query that can be a word or a sentence. Form the query on the basis of index information as follows:
{index_info}

The question is:
{{question}}"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question", "index_info"], template=CYPHER_GENERATION_TEMPLATE
)

In [ ]:
index_info

In [ ]:
chain = GraphCypherQAChain.from_llm(
    llm=ChatOpenAI(temperature=0, model='gpt-4-1106-preview'), cypher_prompt= CYPHER_GENERATION_PROMPT, graph=graph, verbose=True, validate_cypher=True, return_intermediate_steps=True
)
result = chain("any titles related to housing?")

In [ ]:
result